<a href="https://colab.research.google.com/github/NiveskZ/KEVIN_MENESES_DDF_TECH_122025/blob/feat%2Fgeracao-dados/notebooks/01_geracao_dados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📊 Geração de Dados Sintéticos

Este notebook tem como objetivo a geração de dados sintéticos para o case técnico da Dadosfera. A inspiração para o cenário de negócio foi uma loja de varejo local de São José dos Campos (SP), especializada em revenda e distribuição de cervejas artesanais e bebidas alcoólicas em geral.

Com mais de 10 anos de história, a empresa busca expandir seus negócios através da implementação de uma plataforma de dados, visando soluções data-driven e a estruturação de uma operação de e-commerce.
Estratégia de Geração:

  - **Bibliotecas**: Utilização da Faker (localidade pt_BR) para geração de dados cadastrais realistas (nomes, endereços e datas).

  - **Catálogo de Produtos*: Baseado em marcas reais (populares e artesanais como Dogma e Roleta Russa), com preços e SKUs condizentes com o mercado físico.

  - **Modelagem de Probabilidade (Pesos)**: Implementação de lógica de pesos para simular a realidade de consumo, onde produtos mainstream possuem maior volumetria de vendas que produtos premium/nicho.

  - **Histórico de Clientes**: Simulação de uma base de clientes acumulada ao longo de 11 anos, permitindo futuras análises de Cohort e fidelidade.

  - **Motor de Vendas e Sazonalidade**: Janela temporal de 3 anos para análise de série temporal.

      - Sazonalidade semanal: aumento no volume de vendas em finais de semana (sexta a domingo).

      - Escolha aleatória ponderada de produtos e clientes.

## 1. Configuração de Catálogos e Pesos


In [1]:
# Instalando o faker
!pip install faker

In [2]:
# Importando bibliotecas
import pandas as pd
import numpy as np
from faker import Faker
import random
from datetime import datetime, timedelta

In [3]:
# Configurações Iniciais
fake = Faker('pt_BR')
Faker.seed(42)
np.random.seed(42)
random.seed(42)

print("🚀 Iniciando geração de dados para o Case varejo ...")

# ==============================================================================
# CATÁLOGO DE PRODUTOS
# ==============================================================================
# Estrutura: Nome, Categoria, Preço Venda, Custo, Peso (Chance de venda)
# Peso alto (100) = Vende muito (Popular)
# Peso baixo (5-10) = Vende menos (Nicho/Caro)

produtos_base = [
    # --- Cervejas Mainstream (Alto Volume) ---
    {'nome': 'Cerveja Skol Pilsen Lata 350ml', 'cat': 'Cerveja Comum', 'preco': 3.49, 'custo': 2.10, 'peso': 70},
    {'nome': 'Cerveja Brahma Chopp Lata 350ml', 'cat': 'Cerveja Comum', 'preco': 3.59, 'custo': 2.20, 'peso': 80},
    {'nome': 'Cerveja Antarctica Boa Lata 350ml', 'cat': 'Cerveja Comum', 'preco': 3.39, 'custo': 2.05, 'peso': 80},
    {'nome': 'Cerveja Heineken Long Neck 330ml', 'cat': 'Cerveja Premium', 'preco': 7.50, 'custo': 4.80, 'peso': 100},
    {'nome': 'Cerveja Spaten Munchen Lata 350ml', 'cat': 'Cerveja Premium', 'preco': 5.29, 'custo': 3.50, 'peso': 90},
    {'nome': 'Cerveja Imperio Puro Malte Lata 350ml', 'cat': 'Cerveja Premium', 'preco': 4.50, 'custo': 2.90, 'peso': 95},

    # --- Tendência: Sem Glúten / Low Carb ---
    {'nome': 'Cerveja Michelob Ultra Lata 350ml', 'cat': 'Cerveja Especial', 'preco': 5.90, 'custo': 3.80, 'peso': 40},
    {'nome': 'Cerveja Stella Artois Pure Gold 330ml', 'cat': 'Cerveja Especial', 'preco': 6.50, 'custo': 4.20, 'peso': 35},
    {'nome': 'Cerveja Bruder Baixa Gastronomia (S/ Glúten) 600ml', 'cat': 'Cerveja Artesanal', 'preco': 22.90, 'custo': 14.00, 'peso': 20},

    # --- Artesanais: Roleta Russa ---
    {'nome': 'Roleta Russa Easy IPA Lata 350ml', 'cat': 'Cerveja Artesanal', 'preco': 5.99, 'custo': 3.50, 'peso': 30},
    {'nome': 'Roleta Russa Imperial IPA Lata 350ml', 'cat': 'Cerveja Artesanal', 'preco': 10.99, 'custo': 7.60, 'peso': 10},
    {'nome': 'Roleta Russa New England IPA Lata 350ml', 'cat': 'Cerveja Artesanal', 'preco': 9.99, 'custo': 6.80, 'peso': 15},
    {'nome': 'Growler Roleta Russa Easy IPA 1L', 'cat': 'Cerveja Artesanal', 'preco': 19.99, 'custo': 13.40, 'peso': 30},

    # --- Artesanais: Imigração ---
    {'nome': 'Cerveja Imigração Pilsen Garrafa 500ml', 'cat': 'Cerveja Artesanal', 'preco': 16.99, 'custo': 10.00, 'peso': 25},
    {'nome': 'Cerveja Imigração Hop Lager Lata 350ml', 'cat': 'Cerveja Artesanal', 'preco': 4.99, 'custo': 3.50, 'peso': 40},
    {'nome': 'Growler Imigração Export 1L', 'cat': 'Cerveja Artesanal', 'preco': 15.99, 'custo': 10.00, 'peso': 25},

    # --- Artesanais: Dogma, Escafandrista e Locais (Alto valor agregado) ---
    {'nome': 'Cerveja Dogma Rizoma Double IPA 473ml', 'cat': 'Cerveja Artesanal', 'preco': 29.99, 'custo': 18.30, 'peso': 20},
    {'nome': 'Cerveja Dogma American IPA (Lata Nova 355ml)', 'cat': 'Cerveja Artesanal', 'preco': 14.99, 'custo': 8.33, 'peso': 35},
    {'nome': 'Cerveja Escafandrista Smoothie Sour 473ml', 'cat': 'Cerveja Artesanal', 'preco': 45.90, 'custo': 30.00, 'peso': 5},
    {'nome': 'Cerveja Three Lions Session IPA (SJC) 473ml', 'cat': 'Cerveja Artesanal', 'preco': 24.00, 'custo': 15.00, 'peso': 15},
    {'nome': 'Cerveja Campinas Forasteira IPA 473ml', 'cat': 'Cerveja Artesanal', 'preco': 21.00, 'custo': 13.50, 'peso': 18},

    # --- Custo Benefício (Wienbier) ---
    {'nome': 'Wienbier 55 Pilsen Lata 710ml', 'cat': 'Cerveja Comum', 'preco': 8.90, 'custo': 5.50, 'peso': 60},
    {'nome': 'Wienbier 58 Chopp de Vinho 710ml', 'cat': 'Coquetel Alcoólico', 'preco': 9.90, 'custo': 6.00, 'peso': 50},

    # --- Importadas Clássicas ---
    {'nome': 'Cerveja Guinness Draught Lata 440ml', 'cat': 'Importada', 'preco': 37.99, 'custo': 28.00, 'peso': 25},
    {'nome': 'Cerveja Paulaner Weissbier 500ml', 'cat': 'Importada', 'preco': 24.90, 'custo': 16.50, 'peso': 12},
    {'nome': 'Cerveja Blue Moon Belgian White 355ml', 'cat': 'Importada', 'preco': 17.90, 'custo': 11.00, 'peso': 15},

    # --- Destilados e Outros ---
    {'nome': 'Whisky Jack Daniels 1L', 'cat': 'Destilados', 'preco': 169.90, 'custo': 120.00, 'peso': 5},
    {'nome': 'Gin Tanqueray 750ml', 'cat': 'Destilados', 'preco': 119.90, 'custo': 85.00, 'peso': 5},
    {'nome': 'Vodka Absolut 1L', 'cat': 'Destilados', 'preco': 89.90, 'custo': 60.00, 'peso': 8},
    {'nome': 'Cachaça Velho Barreiro 910ml', 'cat': 'Destilados', 'preco': 14.90, 'custo': 9.00, 'peso': 25},

    # --- Não Alcoólicos e Mercearia ---
    {'nome': 'Refrigerante Coca-Cola 2L', 'cat': 'Não Alcoólico', 'preco': 10.90, 'custo': 7.50, 'peso': 100},
    {'nome': 'Água Mineral s/ Gás 500ml', 'cat': 'Não Alcoólico', 'preco': 3.00, 'custo': 0.90, 'peso': 80},
    {'nome': 'Energético Red Bull 250ml', 'cat': 'Não Alcoólico', 'preco': 8.90, 'custo': 5.50, 'peso': 60},
    {'nome': 'Gelo Filtrado Cubo 5kg', 'cat': 'Gelo/Carvão', 'preco': 14.00, 'custo': 6.00, 'peso': 75},
    {'nome': 'Carvão Vegetal 4kg', 'cat': 'Gelo/Carvão', 'preco': 22.00, 'custo': 12.00, 'peso': 40},
    {'nome': 'Chocolate KitKat', 'cat': 'Bomboniere', 'preco': 4.50, 'custo': 2.50, 'peso': 50},
    {'nome': 'Salgadinho Doritos 140g', 'cat': 'Bomboniere', 'preco': 12.90, 'custo': 8.00, 'peso': 45},
    {'nome': 'Arroz Tio João 1kg', 'cat': 'Mercearia', 'preco': 7.90, 'custo': 5.00, 'peso': 20}
]

# Preparando Dataframe de Produtos e vetores de probabilidade
df_produtos = pd.DataFrame(produtos_base)
df_produtos['id_produto'] = range(1, len(df_produtos) + 1)
pesos_produtos = df_produtos['peso'].values / df_produtos['peso'].sum() # Normalizar para probabilidade

produtos_dict = df_produtos.to_dict('records')

print(f"✅ Catálogo criado com {len(df_produtos)} SKUs.")

🚀 Iniciando geração de dados para o Case varejo ...
✅ Catálogo criado com 38 SKUs.


## 2. Geração da Base de Clientes (Histórico de 11 anos)

In [4]:
print("Gerando carteira de clientes...")
num_clientes = 2500

clientes_list = []
for _ in range(num_clientes):
    # Data de cadastro aleatória nos últimos 11 anos
    data_cadastro = fake.date_between(start_date='-11y', end_date='today')

    clientes_list.append({
        'id_cliente': fake.unique.random_number(digits=6),
        'nome': fake.name(),
        'cidade': 'São José dos Campos', # Fixar cidade local
        'bairro': fake.bairro() if random.random() > 0.1 else 'Centro',
        'data_cadastro': data_cadastro,
        'email': fake.email()
    })

df_clientes = pd.DataFrame(clientes_list)

clientes_ids = df_clientes['id_cliente'].values

print(f"✅ Carteira criada com {len(df_clientes)} clientes.")

Gerando carteira de clientes...
✅ Carteira criada com 2500 clientes.


## 3. Processamento do Loop Transacional

In [5]:
print("Gerando transações de vendas com metas de faturamento e feriados...")

import dateutil.easter as easter

def get_special_dates(year):
    pasc = easter.easter(year)
    mae = pasc + timedelta(days=21) # Aproximação: 2º dom de maio costuma ser 3-4 semanas após pascoa (ajuste manual se necessário)
    # Para fins de simulação, vamos definir datas aproximadas para os domingos comemorativos:
    pasc_dom = pasc
    maes_dom = datetime(year, 5, 8) + timedelta(days=(6 - datetime(year, 5, 8).weekday()) % 7)
    pais_dom = datetime(year, 8, 8) + timedelta(days=(6 - datetime(year, 8, 8).weekday()) % 7)
    return {'pascoa': pasc_dom, 'maes': maes_dom, 'pais': pais_dom}

data_inicio_vendas = datetime.now() - timedelta(days=365*3)
dias_simulados = 365 * 3
vendas_list = []
id_venda_counter = 1

ticket_medio_estimado = df_produtos['preco'].mean() * 6 # Estimativa de gasto por venda

for dia in range(dias_simulados):
    data_atual = data_inicio_vendas + timedelta(days=dia)
    especiais = get_special_dates(data_atual.year)

    # --- DEFINIÇÃO DE METAS E HORÁRIOS ---
    faturamento_alvo = 5000 # Base dia de semana
    hora_abertura, hora_fechamento = 9, 20

    dia_semana = data_atual.weekday()

    # Sabado (Meta 24k)
    if dia_semana == 5:
        faturamento_alvo = 24000
        hora_abertura, hora_fechamento = 8, 21

    # Domingo (Meta 10k)
    elif dia_semana == 6:
        faturamento_alvo = 10000
        hora_abertura, hora_fechamento = 8, 14

        # Feriados Móveis no Domingo (Meta igual sábado)
        if data_atual.date() in [especiais['pascoa'], especiais['maes'], especiais['pais']]:
            faturamento_alvo = 24000

    # Natal e Ano Novo (Vésperas 24/12 e 31/12)
    if (data_atual.month == 12 and data_atual.day == 24) or (data_atual.month == 12 and data_atual.day == 31):
        faturamento_alvo = random.randint(60000, 70000)
        hora_abertura, hora_fechamento = 8, 18

    # --- EXECUÇÃO DAS VENDAS ---
    # Calculamos o número de vendas baseado no faturamento alvo / ticket médio
    num_vendas = int(faturamento_alvo / ticket_medio_estimado)
    num_vendas = int(num_vendas * random.uniform(0.9, 1.1)) # Variabilidade de 10%

    produtos_escolhidos = np.random.choice(produtos_dict, size=num_vendas, p=pesos_produtos)
    clientes_do_dia = np.random.choice(clientes_ids, size=num_vendas)

    for i in range(num_vendas):
        produto = produtos_escolhidos[i]
        cat = produto['cat']

        # Ajuste de quantidade por categoria
        if cat == 'Cerveja Comum':
            qtd = random.choices([6, 12, 15, 24], weights=[40, 30, 20, 10], k=1)[0]
        elif cat == 'Cerveja Premium':
            qtd = random.choices([4, 6, 8, 12], weights=[30, 40, 20, 10], k=1)[0]
        else:
            qtd = random.randint(1, 3)

        vendas_list.append({
            'id_venda': id_venda_counter,
            'data_venda': data_atual.strftime("%Y-%m-%d"),
            'hora_venda': f"{random.randint(hora_abertura, hora_fechamento-1):02d}:{random.randint(0, 59):02d}",
            'id_cliente': clientes_do_dia[i],
            'id_produto': produto['id_produto'],
            'quantidade': qtd,
            'valor_unitario': produto['preco'],
            'valor_total': round(qtd * produto['preco'], 2)
        })
        id_venda_counter += 1

df_vendas = pd.DataFrame(vendas_list)

print(f"✅ Dataframe de vendas criado.")

Gerando transações de vendas com metas de faturamento e feriados...
✅ Dataframe de vendas criado.


## 4. Exportação dos Dados e Resumo Final

In [6]:
print("\n--- RESUMO FINAL ---")
print(f"Total Vendas Geradas: {len(df_vendas)}")
print(f"Data Inicial: {df_vendas['data_venda'].min()}")
print(f"Data Final: {df_vendas['data_venda'].max()}")
print(f"Faturamento Total Simulado: R$ {df_vendas['valor_total'].sum():,.2f}")

if len(df_vendas) > 100000:
    print("✅ REQUISITO DO CASE ATENDIDO: > 100.000 Registros")
else:
    print("⚠️ Aumente o período ou vendas diárias.")

# Salvando
df_produtos.drop(columns=['peso']).to_csv('produtos.csv', index=False) # Removemos a coluna peso pois é só lógica interna
df_clientes.to_csv('clientes.csv', index=False)
df_vendas.to_csv('vendas.csv', index=False)

print("\n📂 Arquivos gerados: produtos.csv, clientes.csv, vendas.csv")


--- RESUMO FINAL ---
Total Vendas Geradas: 69044
Data Inicial: 2023-01-01
Data Final: 2025-12-30
Faturamento Total Simulado: R$ 2,418,788.11
⚠️ Aumente o período ou vendas diárias.

📂 Arquivos gerados: produtos.csv, clientes.csv, vendas.csv


In [7]:
# Gerando amostras (10 linhas) para documentação no Git
df_produtos.head(1000).to_csv('produtos_sample.csv', index=False)
df_clientes.head(1000).to_csv('clientes_sample.csv', index=False)
df_vendas.head(1000).to_csv('vendas_sample.csv', index=False)

print("Amostras geradas!")

Amostras geradas!
